# Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as  tf
import numpy as np
import itertools
from keras.optimizers import Adam
from keras.models import clone_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from functools import partial
import itertools
import random
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
from tqdm import tqdm
import cv2

# Definitions

In [ ]:
pwd

'/content'

In [ ]:
base_path = "/content/drive/My Drive/Investigacion/UTN/GIAR/Dataset/cropped_cells_full_v4"

SHAPE = (128,128,1)
batch_size = 300
validation_split = 0.2

In [ ]:
ON_RAM = False

# Load dataset

In [ ]:

if ON_RAM:

    images = np.zeros((len(os.listdir(base_path)), SHAPE[0], SHAPE[1]))
    for idx, file in tqdm(enumerate(os.listdir(base_path)), total=len(os.listdir(base_path))):
        image = cv2.imread(os.path.join(base_path, file), cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (SHAPE[0], SHAPE[1]))
        image = np.array(image) #

        images[idx, : ,:] = image


    from sklearn.model_selection import train_test_split
    x_train, x_test = train_test_split(images, test_size=validation_split, random_state=42)
    x_train.shape

else:

    image_paths = list(tf.io.gfile.glob(base_path + '/*.jpg'))

    # Train / val split
    val_size = int(0.2 * len(image_paths))  # 20% para validación
    train_paths = image_paths[:-val_size]
    val_paths = image_paths[-val_size:]

    train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
    val_dataset = tf.data.Dataset.from_tensor_slices(val_paths)

    # Load & augmentation
    def load_image(path):

      image = tf.io.read_file(path)
      image = tf.image.decode_png(image, channels=1)  # Escala de grises
      image = tf.image.resize(image, (SHAPE[0], SHAPE[1]))
      # Augmentación

      image = tf.image.random_flip_left_right(image)
      image = tf.image.random_flip_up_down(image)
      image = tf.image.random_brightness(image, max_delta=0.1)
      image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
      scales = tf.random.uniform([], 0.8, 1.0)
      crop_size = tf.cast(scales * SHAPE[0], tf.int32)
      image = tf.image.random_crop(image, size=[crop_size, crop_size, 1])
      image = tf.image.resize(image, (SHAPE[0], SHAPE[1]))
      # Normalize [0, 1]
      image = tf.cast(image, tf.float32) / 255.0
      return image,image

    # Train dataset
    train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
    train_dataset = train_dataset.map(lambda x: load_image(x), num_parallel_calls=tf.data.AUTOTUNE)
    train_dataset = train_dataset.shuffle(buffer_size=10000)
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
    train_dataset = train_dataset.repeat()

    # Val dataset
    validation_dataset = val_dataset.map(lambda x: load_image(x), num_parallel_calls=tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.batch(batch_size)
    validation_dataset = validation_dataset.prefetch(tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.repeat()



In [ ]:
len(train_paths)

41307

# Weigthed Loss: SSIM + MAE

In [ ]:
class CustomLoss(tf.keras.losses.Loss):
    def __init__(self, y, z, name="custom_loss"):
        super().__init__(name=name)
        # Weights
        self.y = y
        self.z = z

        #MAE
        self.mae_loss_fn = tf.keras.losses.MeanAbsoluteError()

    # SSIM loss
    def ssim_loss(self, y_true, y_pred):

        ssim = (1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))) / 2
        return ssim

    def call(self, y_true, y_pred):

        mae_loss = self.mae_loss_fn(y_true, y_pred)
        ssim = self.ssim_loss(y_true, y_pred)


        return  self.y * mae_loss + self.z * ssim

# AutoEncoder

In [ ]:
kernel_size = (4,4)
filter = 32

# Encoder
encoder_input = layers.Input(shape=(128, 128, 1))

x = encoder_input

x = layers.Rescaling(1.0 / 255.0)(x)


x = layers.Conv2D(filter, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2, 2), padding='same')(x)

x = layers.Conv2D(filter//2, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2, 2), padding='same')(x)

x = layers.Conv2D(filter//4, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2, 2), padding='same')(x)

x = layers.Conv2D(filter//8, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2, 2), padding='same')(x)

x = layers.Conv2D(filter//16, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D((2, 2), padding='same')(x)

x = layers.Flatten()(x)

encoder_output = x


encoder = Model(encoder_input, encoder_output)

# Decoder

decoder_input = layers.Input(shape=[encoder.output_shape[-1]])

x = decoder_input

x = layers.Reshape((2*filter//16, 2*filter//16, filter//16))(x)


x = layers.Conv2D(filter//16, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(filter//8, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(filter//4, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(filter//2, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(filter, kernel_size, activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2D(1, kernel_size, activation='sigmoid', padding='same')(x)

#x = layers.Rescaling(255.0)(x)

decoder_output = x

decoder = Model(decoder_input, decoder_output)

# Build the autoencoder model
# autoencoder = Model(encoder_input, decoder_output)
autoencoder = Sequential([
    encoder,
    decoder
])

In [ ]:
encoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 128, 128, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling (Rescaling)                │ (None, 128, 128, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 128, 128, 32)        │             544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 16)          │           8,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 64, 64, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 8)           │           2,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 32, 32, 8)           │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 16, 16, 8)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 16, 16, 4)           │             516 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 16, 16, 4)           │              16 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 8, 8, 4)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 8, 8, 2)             │             130 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 8, 8, 2)             │               8 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 4, 4, 2)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 32)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,702 (45.71 KB)

 Trainable params: 11,578 (45.23 KB)

 Non-trainable params: 124 (496.00 B)

In [ ]:
decoder.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 4, 4, 2)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 4, 4, 2)             │              66 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 4, 4, 2)             │               8 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d (UpSampling2D)         │ (None, 8, 8, 2)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 8, 8, 4)             │             132 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 8, 8, 4)             │              16 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_1 (UpSampling2D)       │ (None, 16, 16, 4)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 16, 16, 8)           │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 16, 16, 8)           │              32 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_2 (UpSampling2D)       │ (None, 32, 32, 8)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 32, 32, 16)          │           2,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 32, 32, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_3 (UpSampling2D)       │ (None, 64, 64, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 64, 64, 32)          │           8,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 64, 64, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_4 (UpSampling2D)       │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 128, 128, 1)         │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,767 (45.96 KB)

 Trainable params: 11,643 (45.48 KB)

 Non-trainable params: 124 (496.00 B)

In [ ]:
# Summary of the model
autoencoder.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ functional (Functional)              │ (None, 32)                  │          11,702 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ functional_1 (Functional)            │ (None, 128, 128, 1)         │          11,767 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,469 (91.68 KB)

 Trainable params: 23,221 (90.71 KB)

 Non-trainable params: 248 (992.00 B)

# Compile & fit - Grid search

In [ ]:
# Weight range from 0.2 to 1
y_values = np.arange(0.2, 1.2, 0.2)
z_values = np.arange(0.2, 1.2, 0.2)

# Combinations of weights
param_combinations = list(itertools.product(y_values,z_values))

random.seed(34)
random.shuffle(param_combinations)

# Normalize weights ---> y + z = 1
def normalize_weights(weights):
    total_sum = sum(weights)
    return tuple(weight / total_sum for weight in weights)

param_combinations = [normalize_weights(weights) for weights in param_combinations]
param_combinations =set(param_combinations) #delete duplicate
print("Total combinations: ", len(param_combinations))
for i,combination in enumerate(param_combinations):
    print(f"Combination {i+1}: ", tuple(map(float, combination)))

Total combinations:  19
Combination 1:  (0.25, 0.7500000000000001)
Combination 2:  (0.37500000000000006, 0.625)
Combination 3:  (0.3333333333333333, 0.6666666666666666)
Combination 4:  (0.7142857142857143, 0.28571428571428575)
Combination 5:  (0.4285714285714286, 0.5714285714285714)
Combination 6:  (0.4444444444444445, 0.5555555555555556)
Combination 7:  (0.5555555555555556, 0.4444444444444445)
Combination 8:  (0.7500000000000001, 0.25)
Combination 9:  (0.5, 0.5)
Combination 10:  (0.28571428571428575, 0.7142857142857143)
Combination 11:  (0.5714285714285714, 0.4285714285714286)
Combination 12:  (0.625, 0.37500000000000006)
Combination 13:  (0.8333333333333334, 0.16666666666666669)
Combination 14:  (0.6000000000000001, 0.4)
Combination 15:  (0.6666666666666666, 0.3333333333333333)
Combination 16:  (0.2, 0.8)
Combination 17:  (0.4, 0.6000000000000001)
Combination 18:  (0.8, 0.2)
Combination 19:  (0.16666666666666669, 0.8333333333333334)


In [ ]:
def evaluate_model(autoencoder, train_gen, val_gen, params, steps_per_epoch, val_steps):
    y, z = [np.round(arr, 2) for arr in params]

    loss_fn = CustomLoss(y=y, z=z)
    model = clone_model(autoencoder)
    model.compile(
        loss=loss_fn,
        optimizer=Adam(learning_rate=1e-3)
    )

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=0
    )

    history=model.fit(
        train_gen,
        steps_per_epoch=steps_per_epoch,
        epochs=20,
        validation_data=val_gen,
        validation_steps=val_steps,
        callbacks=[early_stop],
        verbose=1
        )

    val_loss = min(history.history['val_loss'])
    return val_loss


best_params = None
best_loss = float('inf')


steps_per_epoch = int(len(train_paths) // batch_size)
steps_per_epoch_val = int(len(val_paths) // batch_size)


# Evaluate Autoencoder for all the combinations of weights
for params in param_combinations:
    try:
        val_loss = evaluate_model(autoencoder, train_dataset, validation_dataset, params, steps_per_epoch, steps_per_epoch_val)
        print(f"Params {params} -> Val Loss: {val_loss}")

        if val_loss < best_loss:
            best_loss = val_loss
            best_params = params
    except Exception as e:
        print(f"{e}")



Epoch 1/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 4422s 26s/step - loss: 0.2465 - val_loss: 0.2115
Epoch 2/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 89s 525ms/step - loss: 0.2003 - val_loss: 0.2089
Epoch 3/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 73s 538ms/step - loss: 0.1967 - val_loss: 0.2074
Epoch 4/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 73s 537ms/step - loss: 0.1936 - val_loss: 0.2044
Epoch 5/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 72s 530ms/step - loss: 0.1905 - val_loss: 0.2905
Epoch 6/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 69s 508ms/step - loss: 0.1897 - val_loss: 0.1734
Epoch 7/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 70s 512ms/step - loss: 0.1884 - val_loss: 0.1520
Epoch 8/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 69s 504ms/step - loss: 0.1875 - val_loss: 0.2870
Epoch 9/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 69s 509ms/step - loss: 0.1876 - val_loss: 0.2362
Epoch 10/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 69s 508ms/step - loss: 0.1858 - val_loss: 0.1823
Epoch 11/20
137/137 ━━━━━━━━━━━━━━━━━━━━ 69s 508ms/step - loss: 0.1859 - val_loss: 0.1789
Epoch 12/20
137/137

In [ ]:
# Resultados finales
print(f"Best Params: {best_params} -> Best Val Loss: {best_loss}")

Best Params: (np.float64(0.8333333333333334), np.float64(0.16666666666666669)) -> Best Val Loss: 0.09027794003486633
